In [1]:
import os

import numpy as np
import torchvision.transforms as T

import model.transforms as transforms

import utils.utils_mediapipe as utils_mediapipe
import utils.utils_plotly as utils_plotly
import utils.utils_unified_format as utils_unified_format
from config import DATA_CONFIG, TRAIN_CONFIG, VISUALIZER_CONFIG

In [2]:
def get_skeleton_data(points: np.ndarray):
    mp_scatter = utils_plotly.get_scatter_3d(
        points,
        # points_colors,
        size=3,
    )

    mp_graph = utils_plotly.get_scatter_3d(
        utils_unified_format.get_unified_graph_pose(points) if points.shape[0] == 18 else utils_unified_format.get_unified_graph_pose_reduced(points),
        # points_colors,
        mode='lines',
        line=dict(color='darkblue', width=2),
    )

    return [mp_scatter, mp_graph]


def vis_data(data, file_path = ''):
    x_range, y_range, z_range = utils_plotly.get_data_ranges(data)

    fig = utils_plotly.create_figure_3d(len(data), **dict(data=data))
    fig.update_layout(
        title=file_path,
        scene_camera=VISUALIZER_CONFIG.scene_camera,
        scene=dict(
            xaxis=dict(range=x_range),
            yaxis=dict(range=y_range),
            zaxis=dict(range=z_range),
            aspectratio=dict(x=1, y=1, z=1),
        ),
    )
    fig.show()

In [3]:
device = 'cpu'

samples_folder = DATA_CONFIG.mediapipe.points_unified_world_filtered_labeled

label_map = TRAIN_CONFIG.gesture_set.label_map
inv_label_map = TRAIN_CONFIG.gesture_set.inv_label_map

In [4]:
to_keep = TRAIN_CONFIG.transforms_params.to_keep
shape_limit = TRAIN_CONFIG.transforms_params.shape_limit

train_transforms = transforms.TrainTransforms(
    to_keep=to_keep,
    shape_limit=shape_limit,
    device=device,
)
test_transforms = transforms.TestTransforms(
    to_keep=to_keep,
    shape_limit=shape_limit,
    device=device,
)

In [5]:
subject = 101
gesture = 'select'
hand = 'left'
trial = 2
frame = 50

file_path = os.path.join(
    samples_folder,
    f'G{subject}_{gesture}_{hand}_trial{trial}.npy'
)

data = utils_mediapipe.load_points(file_path)

Init data

In [6]:
transform = T.Compose([
    
])

points = np.copy(data[frame, :-1])
points = transform(points[None, ...]).reshape(-1, 3)

points[10] = points[9]
points[13] = points[12]

scatters = get_skeleton_data(points)
vis_data(scatters, file_path)

Transformed

In [7]:
transform = T.Compose([
    transforms.FilterIndex(to_keep),
    transforms.NumpyToTensor(device=device),
    transforms.NormalizeBox(dim=1),
    # transforms.NormalRandom(std=0.05),
])

points = np.copy(data[frame, :-1])
points = transform(points[None, ...]).reshape(-1, 3)

scatters = get_skeleton_data(points)
vis_data(scatters, file_path)

Animation

In [8]:
transform = T.Compose([
    transforms.FilterIndex(to_keep),
    transforms.SO3Random(0, np.pi/2, 0),
    transforms.NumpyToTensor(device=device),
    transforms.NormalizeBox(dim=1),
])

go_frames = []
for frame in range(len(data)):
    points = np.copy(data[frame, :-1])
    points = transform(points[None, ...]).reshape(-1, 3)

    scatters = get_skeleton_data(points)
    go_frame = utils_plotly.get_frame(data=scatters, frame_num=frame)
    go_frames.append(go_frame)

x_range, y_range, z_range = utils_plotly.get_data_ranges([frame.data[0] for frame in go_frames])

fig = utils_plotly.create_figure_3d(2)
fig.update(frames=go_frames)
fig.update_layout(
    title=file_path,
    scene=dict(
        xaxis=dict(range=x_range),
        yaxis=dict(range=y_range),
        zaxis=dict(range=z_range),
        aspectratio=dict(x=1, y=1, z=1),
    ),
    scene_camera=VISUALIZER_CONFIG.scene_camera,
    updatemenus=[VISUALIZER_CONFIG.update_buttons],
    uirevision=True,
)
fig.show(renderer='browser')